<a href="https://colab.research.google.com/github/juancuevas-ops/inteligencia-artificial/blob/main/structured_data_regression_juan_cuevasv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install autokeras
#  Ejecucion y Traduccion por juan cuevas vasquez

     |████████████████████████████████| 162 kB 11.9 MB/s 
     |████████████████████████████████| 133 kB 32.3 MB/s 
     |████████████████████████████████| 462 kB 19.5 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.datasets import fetch_california_housing

import autokeras as ak


## Un ejemplo sencillo
El primer paso es preparar sus datos. Aquí usamos la [vivienda de California
conjunto de datos]((
https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset)
as an example.


In [ ]:

house_dataset = fetch_california_housing()
df = pd.DataFrame(
    np.concatenate(
        (house_dataset.data, house_dataset.target.reshape(-1, 1)), axis=1
    ),
    columns=house_dataset.feature_names + ["Price"],
)
train_size = int(df.shape[0] * 0.9)
df[:train_size].to_csv("train.csv", index=False)
df[train_size:].to_csv("eval.csv", index=False)
train_file_path = "train.csv"
test_file_path = "eval.csv"


El segundo paso es ejecutar el
[Regresor de datos estructurados] (/regresor_de_datos_estructurados).
Como demostración rápida, configuramos las épocas en 10.
También puede dejar las épocas sin especificar para un número adaptativo de épocas.


In [ ]:
# Initialize the structured data regressor.
reg = ak.StructuredDataRegressor(
    overwrite=True, max_trials=3
)  # It tries 3 different models.
# Feed the structured data regressor with training data.
reg.fit(
    # The path to the train.csv file.
    train_file_path,
    # The name of the label column.
    "Price",
    epochs=10,
)
# Predict with the best model.
predicted_y = reg.predict(test_file_path)
# Evaluate the best model with testing data.
print(reg.evaluate(test_file_path, "Price"))


Trial 3 Complete [00h 00m 16s]
val_loss: 0.8531402349472046

Best val_loss So Far: 0.8531402349472046
Total elapsed time: 00h 01m 11s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
581/581 [==============================] - 1s 2ms/step - loss: 1.1905 - mean_squared_error: 1.1905
Epoch 2/10
581/581 [==============================] - 1s 2ms/step - loss: 0.6074 - mean_squared_error: 0.6074
Epoch 3/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5466 - mean_squared_error: 0.5466
Epoch 4/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5254 - mean_squared_error: 0.5254
Epoch 5/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5125 - mean_squared_error: 0.5125
Epoch 6/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5051 - mean_squared_error: 0.5051
Epoch 7/10
581/581 [==============================] - 1s 2ms/step - loss: 0.4987 - mean_squared_error: 0.4987
Epoch 8/10
581/581 [==============================] - 1s 2

## Formato de datos
El AutoKeras StructuredDataRegressor es bastante flexible para el formato de datos.

El ejemplo anterior muestra cómo usar los archivos CSV directamente. Además de los archivos CSV, también admite numpy.ndarray, pandas.DataFrame o tf.data.Dataset. Los datos deben ser bidimensionales con valores numéricos o categóricos.

Para los objetivos de regresión, debe ser un vector de valores numéricos. AutoKeras acepta numpy.ndarray, pandas.DataFrame o pandas.Series.

Los siguientes ejemplos muestran cómo se pueden preparar los datos con numpy.ndarray, pandas.DataFrame y tensorflow.data.Dataset

In [ ]:

# x_train as pandas.DataFrame, y_train as pandas.Series
x_train = pd.read_csv(train_file_path)
print(type(x_train))  # pandas.DataFrame
y_train = x_train.pop("Price")
print(type(y_train))  # pandas.Series

# You can also use pandas.DataFrame for y_train.
y_train = pd.DataFrame(y_train)
print(type(y_train))  # pandas.DataFrame

# You can also use numpy.ndarray for x_train and y_train.
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
print(type(x_train))  # numpy.ndarray
print(type(y_train))  # numpy.ndarray

# Preparing testing data.
x_test = pd.read_csv(test_file_path)
y_test = x_test.pop("Price")

# It tries 10 different models.
reg = ak.StructuredDataRegressor(max_trials=3, overwrite=True)
# Feed the structured data regressor with training data.
reg.fit(x_train, y_train, epochs=10)
# Predict with the best model.
predicted_y = reg.predict(x_test)
# Evaluate the best model with testing data.
print(reg.evaluate(x_test, y_test))


Trial 3 Complete [00h 00m 15s]
val_loss: 1.0982997417449951

Best val_loss So Far: 0.8966108560562134
Total elapsed time: 00h 00m 48s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
581/581 [==============================] - 1s 2ms/step - loss: 1.2364 - mean_squared_error: 1.2364
Epoch 2/10
581/581 [==============================] - 1s 2ms/step - loss: 0.6326 - mean_squared_error: 0.6326
Epoch 3/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5615 - mean_squared_error: 0.5615
Epoch 4/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5320 - mean_squared_error: 0.5320
Epoch 5/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5165 - mean_squared_error: 0.5165
Epoch 6/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5067 - mean_squared_error: 0.5067
Epoch 7/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5004 - mean_squared_error: 0.5004
Epoch 8/10
581/581 [==============================] - 1s 2

El siguiente código muestra cómo convertir numpy.ndarray a tf.data.Dataset.


In [ ]:
train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test))

reg = ak.StructuredDataRegressor(max_trials=3, overwrite=True)
# Feed the tensorflow Dataset to the regressor.
reg.fit(train_set, epochs=10)
# Predict with the best model.
predicted_y = reg.predict(test_set)
# Evaluate the best model with testing data.
print(reg.evaluate(test_set))


Trial 3 Complete [00h 00m 30s]
val_loss: 2.1443941593170166

Best val_loss So Far: 0.814704418182373
Total elapsed time: 00h 01m 26s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
581/581 [==============================] - 2s 3ms/step - loss: 1.1080 - mean_squared_error: 1.1080
Epoch 2/10
581/581 [==============================] - 2s 3ms/step - loss: 0.6221 - mean_squared_error: 0.6221
Epoch 3/10
581/581 [==============================] - 2s 3ms/step - loss: 0.5594 - mean_squared_error: 0.5594
Epoch 4/10
581/581 [==============================] - 2s 3ms/step - loss: 0.5418 - mean_squared_error: 0.5418
Epoch 5/10
581/581 [==============================] - 2s 3ms/step - loss: 0.5165 - mean_squared_error: 0.5165
Epoch 6/10
581/581 [==============================] - 2s 3ms/step - loss: 0.5061 - mean_squared_error: 0.5061
Epoch 7/10
581/581 [==============================] - 2s 3ms/step - loss: 0.4981 - mean_squared_error: 0.4981
Epoch 8/10
581/581 [==============================] - 2s 3m

También puede especificar los nombres y tipos de columna para los datos de la siguiente manera. Él
`column_names` es opcional si los datos de entrenamiento ya tienen los nombres de las columnas,
p.ej. pandas.DataFrame, archivo CSV. Cualquier columna, cuyo tipo no se especifica será
deducirse de los datos de entrenamiento.


In [ ]:
# Initialize the structured data regressor.
reg = ak.StructuredDataRegressor(
    column_names=[
        "MedInc",
        "HouseAge",
        "AveRooms",
        "AveBedrms",
        "Population",
        "AveOccup",
        "Latitude",
        "Longitude",
    ],
    column_types={"MedInc": "numerical", "Latitude": "numerical"},
    max_trials=10,  # It tries 10 different models.
    overwrite=True,
)



## Datos de validación
De forma predeterminada, AutoKeras utiliza el último 20 % de los datos de entrenamiento como datos de validación. Como
como se muestra en el ejemplo a continuación, puede usar `validation_split` para especificar el
porcentaje.


In [ ]:
reg.fit(
    x_train,
    y_train,
    # Split the training data and use the last 15% as validation data.
    validation_split=0.15,
    epochs=10,
)


Trial 10 Complete [00h 00m 19s]
val_loss: 3.1975913047790527

Best val_loss So Far: 0.759193480014801
Total elapsed time: 00h 02m 38s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
581/581 [==============================] - 2s 2ms/step - loss: 0.9416 - mean_squared_error: 0.9416
Epoch 2/10
581/581 [==============================] - 1s 2ms/step - loss: 0.6174 - mean_squared_error: 0.6174
Epoch 3/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5794 - mean_squared_error: 0.5794
Epoch 4/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5619 - mean_squared_error: 0.5619
Epoch 5/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5457 - mean_squared_error: 0.5457
Epoch 6/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5365 - mean_squared_error: 0.5365
Epoch 7/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5280 - mean_squared_error: 0.5280
Epoch 8/10
581/581 [==============================] - 1s 2

También puede usar su propio conjunto de validación
en lugar de dividirlo de los datos de entrenamiento con `validation_data`.


In [ ]:
split = 500
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]
reg.fit(
    x_train,
    y_train,
    # Use your own validation set.
    validation_data=(x_val, y_val),
    epochs=10,
)


## Espacio de búsqueda personalizado
Para usuarios avanzados, puede personalizar su espacio de búsqueda usando
[AutoModel](/auto_model/#automodel-class) en lugar de
[Regresor de datos estructurados] (/regresor_de_datos_estructurados). Puede configurar el
[StructuredDataBlock](/block/#structureddatablock-class) para algunos de alto nivel
configuraciones, por ejemplo, `categorical_encoding` para saber si usar el
[CategoricalToNumerical](/block/#categoricaltonumerical-class). también puedes hacer
no especificar estos argumentos, lo que dejaría las diferentes opciones para ser
sintonizado automáticamente. Vea el siguiente ejemplo para más detalles.


In [ ]:

input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=True)(input_node)
output_node = ak.RegressionHead()(output_node)
reg = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=3
)
reg.fit(x_train, y_train, epochs=10)


Trial 3 Complete [00h 00m 02s]
val_loss: nan

Best val_loss So Far: 2.8545517921447754
Total elapsed time: 00h 00m 06s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
16/16 [==============================] - 0s 2ms/step - loss: 24181.7188 - mean_squared_error: 24181.7188
Epoch 2/10
16/16 [==============================] - 0s 2ms/step - loss: 8841.2930 - mean_squared_error: 8841.2930
Epoch 3/10
16/16 [==============================] - 0s 2ms/step - loss: 6864.2402 - mean_squared_error: 6864.2402
Epoch 4/10
16/16 [==============================] - 0s 2ms/step - loss: 5135.8081 - mean_squared_error: 5135.8081
Epoch 5/10
16/16 [==============================] - 0s 2ms/step - loss: 4264.8076 - mean_squared_error: 4264.8076
Epoch 6/10
16/16 [==============================] - 0s 2ms/step - loss: 3286.9932 - mean_squared_error: 3286.9932
Epoch 7/10
16/16 [==============================] - 0s 2ms/step - loss: 2666.2947 - mean_squared_error: 2666.2947
Epoch 8/10
16/16 [=========================

El uso de [AutoModel](/auto_model/#automodel-class) es similar al
[API funcional](https://www.tensorflow.org/guide/keras/funcional) de Keras.
Básicamente, estás construyendo un gráfico, cuyos bordes son bloques y los nodos son
salidas intermedias de bloques. Para agregar un borde de `input_node` a
`output_node` con `output_node = ak.[some_block]([block_args])(input_node)`.

Incluso puede usar bloques de grano más fino para personalizar el espacio de búsqueda
Aún más. Vea el siguiente ejemplo.


In [ ]:

input_node = ak.StructuredDataInput()
output_node = ak.CategoricalToNumerical()(input_node)
output_node = ak.DenseBlock()(output_node)
output_node = ak.RegressionHead()(output_node)
reg = ak.AutoModel(
    inputs=input_node, outputs=output_node, max_trials=3, overwrite=True
)
reg.fit(x_train, y_train, epochs=10)


Trial 3 Complete [00h 00m 02s]
val_loss: 1.5638854503631592

Best val_loss So Far: 1.367082118988037
Total elapsed time: 00h 00m 06s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
16/16 [==============================] - 0s 2ms/step - loss: 288.5348 - mean_squared_error: 288.5348
Epoch 2/10
16/16 [==============================] - 0s 2ms/step - loss: 22.7222 - mean_squared_error: 22.7222
Epoch 3/10
16/16 [==============================] - 0s 2ms/step - loss: 5.7583 - mean_squared_error: 5.7583
Epoch 4/10
16/16 [==============================] - 0s 2ms/step - loss: 1.8155 - mean_squared_error: 1.8155
Epoch 5/10
16/16 [==============================] - 0s 3ms/step - loss: 4.5803 - mean_squared_error: 4.5803
Epoch 6/10
16/16 [==============================] - 0s 3ms/step - loss: 1.6187 - mean_squared_error: 1.6187
Epoch 7/10
16/16 [==============================] - 0s 2ms/step - loss: 3.4958 - mean_squared_error: 3.4958
Epoch 8/10
16/16 [==============================] - 0s 2ms/step - l

In [ ]:
model = reg.export_model()
model.summary()
# numpy array in object (mixed type) is not supported.
# you need convert it to unicode or float first.
model.predict(x_train)



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8)]               0         
                                                                 
 multi_category_encoding (Mu  (None, 8)                0         
 ltiCategoryEncoding)                                            
                                                                 
 dense (Dense)               (None, 32)                288       
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1024)              33792     
                                                                 
 re_lu_1 (ReLU)              (None, 1024)              0         
                                                             

array([[ 2.8247871 ],
       [10.773115  ],
       [ 3.7223537 ],
       [ 3.7450383 ],
       [ 3.7485516 ],
       [ 3.230241  ],
       [ 4.942045  ],
       [ 5.1140957 ],
       [ 5.1890335 ],
       [ 6.6395035 ],
       [ 4.7104893 ],
       [ 7.0785484 ],
       [ 5.3581076 ],
       [ 2.7166507 ],
       [ 5.8022976 ],
       [ 3.9422004 ],
       [ 4.384858  ],
       [ 3.8760917 ],
       [ 4.5742307 ],
       [ 3.8297055 ],
       [ 2.7668884 ],
       [ 4.50236   ],
       [ 4.709077  ],
       [ 4.2544985 ],
       [ 4.7046366 ],
       [ 2.521788  ],
       [ 3.6807983 ],
       [ 5.3248425 ],
       [ 4.9145794 ],
       [ 2.8597884 ],
       [ 4.1860666 ],
       [ 5.1025867 ],
       [ 4.583309  ],
       [ 4.091081  ],
       [ 6.015973  ],
       [ 3.55669   ],
       [ 4.8130665 ],
       [ 4.543956  ],
       [ 4.0728197 ],
       [ 6.4747562 ],
       [ 4.708795  ],
       [ 3.4414237 ],
       [ 2.9376903 ],
       [ 3.1755698 ],
       [ 3.9787877 ],
       [ 4

## Reference
[StructuredDataRegressor](/structured_data_regressor),
[AutoModel](/auto_model/#automodel-class),
[StructuredDataBlock](/block/#structureddatablock-class),
[DenseBlock](/block/#denseblock-class),
[StructuredDataInput](/node/#structureddatainput-class),
[RegressionHead](/block/#regressionhead-class),
[CategoricalToNumerical](/block/#categoricaltonumerical-class).
